In [ ]:
import sqlite3, pandas as pd, os, string as str, re, requests, time, tkinter as tk
from bs4 import BeautifulSoup
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()

root.attributes('-topmost', 0)
file_path = filedialog.askopenfilename(title="Select File")
root.attributes('-topmost', 1)

file_path = file_path.replace('\'','/')
folder_path_value = file_path.rfind('/') + 1
folder_path = file_path[:folder_path_value]

dataframe = pd.read_csv(file_path)

df = dataframe

In [ ]:
df = df.fillna('')
filtered_df = df[df['Title'] == '']
max_count = filtered_df['Link'].count() - 1
current_count = 0

url_link = filtered_df['Link'].values[current_count]
job_location_pattern = re.compile(r'in (.*?) \B', re.I|re.S)
job_title_pattern = re.compile(r'hiring (.*?) in', re.I|re.S)

headers = {'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}
session = requests.Session()

def html_grab(url_link):
    r = requests.get(url_link, headers = headers)
    html_source = r.text
    html_soup = BeautifulSoup(html_source,'html.parser')
    return html_soup

def requests_response(url_link):
    r = requests.get(url_link, headers = headers)
    r.status_code
    if r.status_code == 404:
        return 'fail'
    else:
        return 'pass'

def html_title_string(html_soup):
    title_string = html_soup.title.string
    company = title_string.split("hiring")[0]
    job_title = re.search(job_title_pattern, title_string)
    job_location = re.search(job_location_pattern, title_string)
    return company.strip(),job_title.group(1).strip(),job_location.group(1).strip()


In [ ]:
while current_count <= max_count:
    print('Currently working on ' + f"{current_count + 1}" + ' out of ' + f"{max_count + 1}" + ' links.', end="\r")
    response_status = requests_response(filtered_df['Link'].values[current_count])
    html_data = html_grab(filtered_df['Link'].values[current_count])
    if response_status == 'pass' and 'hiring' in html_data.title.string:        
        data_set = html_title_string(html_data)
        df.loc[current_count,'Company'] = data_set[0]
        df.loc[current_count,'Title'] = data_set[1]
        df.loc[current_count,'Location'] = data_set[2]
    else:
        df.loc[current_count,'Notes'] = 'invalid link or expired post'
        time.sleep(3)
    current_count = current_count + 1

df.to_excel(folder_path+'updated_linkedin_links.xlsx', index=False)
print('Job completed.')